# 学習

In [ ]:
# ライセンス情報とコピーライト表示
# Copyright (C) 2023, Inria
# GRAPHDECO research group, https://team.inria.fr/graphdeco
# All rights reserved.
#
# This software is free for non-commercial, research and evaluation use 
# under the terms of the LICENSE.md file.
#
# For inquiries contact  george.drettakis@inria.fr
#

# 必要なモジュールのインポート
import os
import torch
from random import randint
from utils.loss_utils import l1_loss, ssim
from gaussian_renderer import render, network_gui
import sys
from scene import Scene, GaussianModel
from utils.general_utils import safe_state
import uuid
from tqdm import tqdm
from utils.image_utils import psnr
from argparse import ArgumentParser, Namespace
from arguments import ModelParams, PipelineParams, OptimizationParams

# TensorBoardのインポートを試みる（オプション）
try:
    from torch.utils.tensorboard import SummaryWriter
    TENSORBOARD_FOUND = True
except ImportError:
    TENSORBOARD_FOUND = False

# メイントレーニング関数
def training(dataset, opt, pipe, testing_iterations, saving_iterations, checkpoint_iterations, checkpoint, debug_from):
    ####################################################################################################
    # 初期設定
    ####################################################################################################
    first_iter = 0
    # 出力ディレクトリとロガーの準備
    tb_writer = prepare_output_and_logger(dataset)
    # ガウシアンモデルの初期化
    gaussians = GaussianModel(dataset.sh_degree)
    # シーンの初期化
    scene = Scene(dataset, gaussians)
    # ガウシアンモデルのトレーニングセットアップ
    gaussians.training_setup(opt)
    # チェックポイントがある場合、モデルを復元
    if checkpoint:
        (model_params, first_iter) = torch.load(checkpoint)
        gaussians.restore(model_params, opt)

    # 背景色の設定
    bg_color = [1, 1, 1] if dataset.white_background else [0, 0, 0]
    background = torch.tensor(bg_color, dtype=torch.float32, device="cuda")

    # CUDAイベントの初期化（タイミング計測用）
    iter_start = torch.cuda.Event(enable_timing = True)
    iter_end = torch.cuda.Event(enable_timing = True)

    # 初期化
    viewpoint_stack = None
    ema_loss_for_log = 0.0
    # プログレスバーの設定
    progress_bar = tqdm(range(first_iter, opt.iterations), desc="Training progress")
    first_iter += 1

    ####################################################################################################
    # メインのトレーニングループ
    # 各イテレーションで以下の処理を行う
    # 1. ネットワークGUIとの接続を試みる
    # 2. 学習率の更新
    # 3. 1000イテレーションごとにSHレベルを増加（最大次数まで）
    # 4. ランダムにカメラを選択
    # 5. 背景色の設定（ランダムまたは固定）
    # 6. シーンのレンダリング
    # 7. 損失の計算と逆伝播
    ####################################################################################################
    for iteration in range(first_iter, opt.iterations + 1):        
        # ネットワークGUIとの接続を試みる
        if network_gui.conn == None:
            network_gui.try_connect()
        # ネットワークGUIが接続された場合の処理
        while network_gui.conn != None:
            try:
                net_image_bytes = None
                # GUIからのコマンドを受信
                custom_cam, do_training, pipe.convert_SHs_python, pipe.compute_cov3D_python, keep_alive, scaling_modifer = network_gui.receive()
                if custom_cam != None:
                    # カスタムカメラでレンダリング
                    net_image = render(custom_cam, gaussians, pipe, background, scaling_modifer)["render"]
                    net_image_bytes = memoryview((torch.clamp(net_image, min=0, max=1.0) * 255).byte().permute(1, 2, 0).contiguous().cpu().numpy())
                # レンダリング結果をGUIに送信
                network_gui.send(net_image_bytes, dataset.source_path)
                if do_training and ((iteration < int(opt.iterations)) or not keep_alive):
                    break
            except Exception as e:
                network_gui.conn = None

        # イテレーション開始時間を記録
        iter_start.record()

        # 学習率の更新
        gaussians.update_learning_rate(iteration)

        # 1000イテレーションごとにSHレベルを増加（最大次数まで）
        if iteration % 1000 == 0:
            gaussians.oneupSHdegree()

        # ランダムにカメラを選択
        if not viewpoint_stack:
            viewpoint_stack = scene.getTrainCameras().copy()
        viewpoint_cam = viewpoint_stack.pop(randint(0, len(viewpoint_stack)-1))

        # デバッグモードの設定
        if (iteration - 1) == debug_from:
            pipe.debug = True

        # 背景色の設定（ランダムまたは固定）
        bg = torch.rand((3), device="cuda") if opt.random_background else background

        # シーンのレンダリング
        render_pkg = render(viewpoint_cam, gaussians, pipe, bg)
        image, viewspace_point_tensor, visibility_filter, radii = render_pkg["render"], render_pkg["viewspace_points"], render_pkg["visibility_filter"], render_pkg["radii"]

        # 損失の計算
        gt_image = viewpoint_cam.original_image.cuda()
        Ll1 = l1_loss(image, gt_image)
        loss = (1.0 - opt.lambda_dssim) * Ll1 + opt.lambda_dssim * (1.0 - ssim(image, gt_image))
        loss.backward()

        # イテレーション終了時間を記録
        iter_end.record()


        with torch.no_grad():
            # プログレスバーの更新
            ema_loss_for_log = 0.4 * loss.item() + 0.6 * ema_loss_for_log
            if iteration % 10 == 0:
                progress_bar.set_postfix({"Loss": f"{ema_loss_for_log:.{7}f}"})
                progress_bar.update(10)
            if iteration == opt.iterations:
                progress_bar.close()

            # ログの記録と保存
            training_report(tb_writer, iteration, Ll1, loss, l1_loss, iter_start.elapsed_time(iter_end), testing_iterations, scene, render, (pipe, background))
            if (iteration in saving_iterations):
                print("\n[ITER {}] Saving Gaussians".format(iteration))
                scene.save(iteration)

            # 密度化処理
            if iteration < opt.densify_until_iter:
                # 画像空間での最大半径を追跡（プルーニング用）
                gaussians.max_radii2D[visibility_filter] = torch.max(gaussians.max_radii2D[visibility_filter], radii[visibility_filter])
                gaussians.add_densification_stats(viewspace_point_tensor, visibility_filter)

                if iteration > opt.densify_from_iter and iteration % opt.densification_interval == 0:
                    size_threshold = 20 if iteration > opt.opacity_reset_interval else None
                    gaussians.densify_and_prune(opt.densify_grad_threshold, 0.005, scene.cameras_extent, size_threshold)
                
                if iteration % opt.opacity_reset_interval == 0 or (dataset.white_background and iteration == opt.densify_from_iter):
                    gaussians.reset_opacity()

            # オプティマイザのステップ
            if iteration < opt.iterations:
                gaussians.optimizer.step()
                gaussians.optimizer.zero_grad(set_to_none = True)

            # チェックポイントの保存
            if (iteration in checkpoint_iterations):
                print("\n[ITER {}] Saving Checkpoint".format(iteration))
                torch.save((gaussians.capture(), iteration), scene.model_path + "/chkpnt" + str(iteration) + ".pth")